In [ ]:
print("Question 2")

Question 2


In [2]:
!pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! pip install -q kaggle

In [5]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [6]:
!cp /content/drive/MyDrive/timepass/Kaggle/kaggle.json ~/.kaggle/kaggle.json

In [7]:
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
 ! kaggle datasets list

ref                                                        title                                         size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------  -------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
nelgiriyewithana/top-spotify-songs-2023                    Most Streamed Spotify Songs 2023              47KB  2023-08-26 11:04:57          12635        407  1.0              
joebeachcapital/students-performance                       Students Performance                           2KB  2023-08-31 00:50:11           4255        112  1.0              
carlmcbrideellis/zzzs-lightweight-training-dataset-target  Zzzs: Lightweight training dataset + target  185MB  2023-09-20 19:52:40            454         68  1.0              
iamsouravbanerjee/airline-dataset                          Airline Dataset                                8MB  2023-09-1

In [9]:
!kaggle datasets download -d robikscube/driving-video-with-object-tracking

100% 18.6G/18.6G [04:48<00:00, 109MB/s]
100% 18.6G/18.6G [04:48<00:00, 69.0MB/s]


In [ ]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip install opencv-python-headless tensorflow

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import zipfile

# Mount Google Drive (if your videos are stored in Google Drive)
from google.colab import drive
drive.mount('/content/drive')

# Path to the pre-trained object detection model
model_path = 'path_to_model_directory'  # Replace with your model path
output_path = '/content/output_videos'   # Output directory for processed videos
zip_file_path = '/content/driving-video-with-object-tracking.zip'  # Path to the ZIP file
unzip_dir = '/content/unzipped_videos'   # Directory to unzip the videos

# Create the output directory if it doesn't exist
os.makedirs(output_path, exist_ok=True)

# Function to preprocess a frame (replace with your preprocessing logic)
def preprocess(frame):
    # Your preprocessing code here
    return frame

# Function to draw bounding boxes (replace with your drawing logic)
def draw_boxes(frame, detections):
    # Your drawing code here
    return frame

# Extract the ZIP file containing videos
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

# List all video files in the unzipped directory
video_files = [f for f in os.listdir(unzip_dir) if f.endswith('.mov')]




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install opencv-python-headless tensorflow pandas pyarrow


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.builders import model_builder
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import label_map_util

# Define paths to your data and model
video_dir = '/content/unzipped_videos/bdd100k_videos_train_00/bdd100k/videos/train'
label_file_csv = '/content/unzipped_videos/mot_labels.csv'
label_file_parquet = '/content/unzipped_videos/mot_labels.parquet'
model_dir = '/content/unzipped_videos'

# Create a label map and load labels (if available)
label_map_path = 'path_to_label_map.pbtxt'  # You'll need to create this file
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=90)
category_index = label_map_util.create_category_index(categories)

# Load your video data
video_files = [f for f in os.listdir(video_dir) if f.endswith('.mov')]

# Load your annotations (labels)
annotations_csv = pd.read_csv(label_file_csv)
annotations_parquet = pd.read_parquet(label_file_parquet)

# Load a pre-trained object detection model
def load_model(model_dir):
    # Load pipeline config and build detection model
    pipeline_config = os.path.join(model_dir, 'pipeline.config')
    configs = config_util.get_configs_from_pipeline_file(pipeline_config)
    model_config = configs['model']
    detection_model = model_builder.build(model_config=model_config, is_training=False)

    # Restore checkpoint
    ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
    ckpt.restore(os.path.join(model_dir, 'checkpoint', 'ckpt-0')).expect_partial()

    return detection_model

detection_model = load_model(model_dir)

# Function to perform object detection on a frame
def detect_objects(frame):
    input_tensor = tf.convert_to_tensor(np.expand_dims(frame, 0))
    detections = detection_model(input_tensor)
    return detections

# Function to draw bounding boxes on a frame
def draw_boxes_on_frame(frame, detections):
    viz_utils.visualize_boxes_and_labels_on_image_array(
        frame,
        detections['detection_boxes'][0].numpy(),
        detections['detection_classes'][0].numpy().astype(np.int32),
        detections['detection_scores'][0].numpy(),
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,  # Adjust as needed
        min_score_thresh=0.30,  # Adjust as needed
        agnostic_mode=False)
    return frame

# Function to process and save a video
def process_and_save_video(input_video_path, output_video_path):
    cap = cv2.VideoCapture(input_video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'XVID'), 30, (frame_width, frame_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Perform object detection on the frame
        detections = detect_objects(frame)

        # Draw bounding boxes on the frame
        frame_with_boxes = draw_boxes_on_frame(frame, detections)

        # Write the frame with bounding boxes to the output video
        out.write(frame_with_boxes)

    cap.release()
    out.release()

# Process and save each video
for video_file in video_files:
    input_video_path = os.path.join(video_dir, video_file)
    output_video_path = os.path.join(output_path, video_file)
    process_and_save_video(input_video_path, output_video_path)


In [ ]:
!pip install opencv-python-headless numpy tensorflow


In [ ]:
import os

output_folder = "/content/output_videos"
os.makedirs(output_folder, exist_ok=True)


In [ ]:
!pip install opencv-python-headless numpy


In [ ]:
import cv2
import numpy as np

# Load YOLOv4 model
net = cv2.dnn.readNet("yolov4.weights", "yolov4.cfg")

# Load class names
classes = []
with open("coco.names", "r") as f:
    classes = f.read().strip().split("\n")

# Input and output paths
input_folder = "/content/unzipped_videos/bdd100k_videos_train_00/bdd100k/videos/train/"
output_folder = "/content/output_videos"

# Process each video
for video_file in os.listdir(input_folder):
    input_video_path = os.path.join(input_folder, video_file)
    output_video_path = os.path.join(output_folder, f"processed_{video_file}")

    cap = cv2.VideoCapture(input_video_path)

    # Get video properties
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(5))

    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
        net.setInput(blob)

        # Perform object detection
        layer_names = net.getUnconnectedOutLayersNames()
        detections = net.forward(layer_names)

        # Process detections and draw bounding boxes (similar to previous example)
        # ...

        # Write the frame with bounding boxes to the output video
        out.write(frame)

    # Release video objects
    cap.release()
    out.release()

# Close all OpenCV windows
cv2.destroyAllWindows()


In [2]:
!pip install ultralytics -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.1/618.1 kB 7.5 MB/s eta 0:00:00


In [10]:
import zipfile

# Specify the path to the ZIP file you want to unzip
zip_file_path = '/content/driving-video-with-object-tracking.zip'

# Specify the directory where you want to extract the contents
extracted_dir_path = '/content/'

# Use the zipfile module to extract the contents
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir_path)


In [11]:
# Install Darknet (adjust based on your YOLO version)
!git clone https://github.com/AlexeyAB/darknet.git
%cd darknet
!make

# Install OpenCV
!pip install opencv-python

Cloning into 'darknet'...
remote: Enumerating objects: 15769, done.
remote: Counting objects: 100% (231/231), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 15769 (delta 117), reused 171 (delta 98), pack-reused 15538
Receiving objects: 100% (15769/15769), 14.67 MiB | 12.04 MiB/s, done.
Resolving deltas: 100% (10547/10547), done.
/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -c ./src/image_opencv.cpp -o obj/image_opencv.o
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -c ./src/http_stream.cpp -o obj/http_stream.o
./src/http_stream.cpp: In member function ‘bool JSON_sender::write(const char*)’:
./src/http_stream.cpp:253:21: warning: unused variable ‘n’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.ht

In [12]:
import os

video_directory = "/content/bdd100k_videos_train_00/bdd100k/videos/train"
video_files = os.listdir(video_directory)
selected_videos = video_files[:20]  # Adjust the slice to select the first 20 videos

# Create a list of full paths to the selected videos
video_paths = [os.path.join(video_directory, video) for video in selected_videos]


In [15]:
import os
import cv2
import subprocess

# Define the path to your YOLO model
model_path = "yolov8m.pt"

# Define the directory containing the videos
video_directory = "/content/bdd100k_videos_train_00/bdd100k/videos/train"

# Define the output directory for processed videos
output_directory = "/content/output_videos"
os.makedirs(output_directory, exist_ok=True)

# List all video files in the directory
video_files = os.listdir(video_directory)

# Select the first 20 videos (adjust as needed)
selected_videos = video_files[:20]

# Loop through the selected videos
for video_file in selected_videos:
    video_path = os.path.join(video_directory, video_file)

    # Define the output video file path
    output_path = os.path.join(output_directory, os.path.basename(video_file))

    # Define the YOLO command
    yolo_command = f"!yolo detect predict model={model_path} source={video_path} output={output_path}"

    # Run the YOLO command using subprocess
    subprocess.run(yolo_command, shell=True)

    # Create a command to add libx264 codec using FFmpeg
    ffmpeg_command = f"!ffmpeg -i {output_path} -vcodec libx264 {os.path.join(output_directory, 'vid_' + os.path.basename(output_path))}"

    # Run the FFmpeg command using subprocess
    subprocess.run(ffmpeg_command, shell=True)


# Release all resources
cv2.destroyAllWindows()


In [16]:
!yolo detect predict model=yolov8m.pt source="/content/drive/MyDrive/timepass/video"

/bin/bash: line 1: yolo: command not found


In [17]:
!pip install ultralytics -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.1/618.1 kB 6.4 MB/s eta 0:00:00


In [18]:
!yolo detect predict model=yolov8m.pt source="/content/drive/MyDrive/timepass/video"

Streaming output truncated to the last 5000 lines.
video 6/23 (950/1211) /content/drive/MyDrive/timepass/video/14.mov: 384x640 6 cars, 1244.0ms
video 6/23 (951/1211) /content/drive/MyDrive/timepass/video/14.mov: 384x640 7 cars, 1367.1ms
video 6/23 (952/1211) /content/drive/MyDrive/timepass/video/14.mov: 384x640 4 cars, 1210.8ms
video 6/23 (953/1211) /content/drive/MyDrive/timepass/video/14.mov: 384x640 4 cars, 829.6ms
video 6/23 (954/1211) /content/drive/MyDrive/timepass/video/14.mov: 384x640 4 cars, 820.1ms
video 6/23 (955/1211) /content/drive/MyDrive/timepass/video/14.mov: 384x640 3 cars, 829.4ms
video 6/23 (956/1211) /content/drive/MyDrive/timepass/video/14.mov: 384x640 3 cars, 809.8ms
video 6/23 (957/1211) /content/drive/MyDrive/timepass/video/14.mov: 384x640 1 car, 791.5ms
video 6/23 (958/1211) /content/drive/MyDrive/timepass/video/14.mov: 384x640 3 cars, 811.0ms
video 6/23 (959/1211) /content/drive/MyDrive/timepass/video/14.mov: 384x640 3 cars, 823.2ms
video 6/23 (960/1211) /cont

In [20]:
!ffmpeg -i {"/content/darknet/runs/detect/predict/1.avi"} -vcodec libx264 {"video1.avi"}
!ffmpeg -i {"/content/darknet/runs/detect/predict/10.avi"} -vcodec libx264 {"video10.avi"}
!ffmpeg -i {"/content/darknet/runs/detect/predict/11.avi"} -vcodec libx264 {"video11.avi"}
!ffmpeg -i {"/content/darknet/runs/detect/predict/12.avi"} -vcodec libx264 {"video12.avi"}
!ffmpeg -i {"/content/darknet/runs/detect/predict/13.avi"} -vcodec libx264 {"video13.avi"}
!ffmpeg -i {"/content/darknet/runs/detect/predict/14.avi"} -vcodec libx264 {"video14.avi"}
!ffmpeg -i {"/content/darknet/runs/detect/predict/15.avi"} -vcodec libx264 {"video15.avi"}
!ffmpeg -i {"/content/darknet/runs/detect/predict/16.avi"} -vcodec libx264 {"video16.avi"}
!ffmpeg -i {"/content/darknet/runs/detect/predict/17.avi"} -vcodec libx264 {"video17.avi"}
!ffmpeg -i {"/content/darknet/runs/detect/predict/18.avi"} -vcodec libx264 {"video18.avi"}

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [21]:
!ffmpeg -i {"/content/darknet/runs/detect/predict/1.avi"} -vcodec libx264 {"video1.avi"}

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [22]:
!ffmpeg -i {"/content/darknet/runs/detect/predict/1.avi"} -vcodec libx264 {"/content/drive/MyDrive/timepass/Output/video1.avi"}

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab